## Libraries

In [ ]:
# data
import numpy as np
import pandas as pd
import xarray as xr

# plotting
import holoviews as hv
from holoviews import opts
import matplotlib.pyplot as plt

# allow "direct" plotting pandas and xarray just in case
import hvplot.pandas  # noqa
import hvplot.xarray  # noqa
pd.options.plotting.backend = 'holoviews'

# setup plotting libs
hv.extension('bokeh', 'matplotlib')
%matplotlib inline

# not necessary but why not
from pandas_profiling import ProfileReport

# repeatability
np.random.seed(123)

# models
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Read and show raw data


In [ ]:
from os import path
raw_data = {k: pd.read_csv('./Data/' + f, sep=',', header=None) for k, f in 
                [('albedo', 'Albedo.csv'), 
                 ('a', 'Element_A_Map.csv'), ('b', 'Element_B_Map.csv'),
                 ('c', 'Element_C_Map.csv'), ('d', 'Element_D_Map.csv')]
           }
for r in raw_data.values():
    r.index.name='y'
    r.columns.name='x'
    r.info()

In [ ]:
data = xr.Dataset(raw_data)
data

In [ ]:
data.albedo.plot() # use hvplot to show data

In [ ]:
from functools import partial

plot_image = partial(hv.Image, kdims=['x', 'y'])

def layout_vars(foo):
    return lambda ds: hv.NdLayout({v: foo(ds[v]) for v in ds}, kdims=['variable']).cols(2)

layout_vars(plot_image)(data).opts(opts.Image(width=len(data.x), height=len(data.y), colorbar=True))

## Is albedo visibly related to elements?

In [ ]:
def gridspace_vars(foo):
    return lambda tidy: hv.GridSpace({
        (a,b): foo(tidy, [a, b])
        for a in tidy.columns
        for b in tidy.columns
    })        


hv.render(
    gridspace_vars(hv.Points) \
        (data.to_dataframe().query('albedo > 0')) \
        .opts(fig_size=256, backend='matplotlib'), # bokeh backend eats browser memory with too many client-side points
    backend='matplotlib'
)

### Too many dots

## Can we draw a better plot?

### What would the profiling panda do?

In [ ]:
ProfileReport(data.to_dataframe().query('albedo > 0').set_index('albedo'))

### HexTiles!

In [ ]:
def grid_diag_vars(pair, solo):
    return lambda tidy: hv.GridSpace({
        (a,b): pair(tidy, [a, b]) if a!=b else solo(tidy, a)
        for a in tidy.columns
        for b in tidy.columns
    })     

def map_datasets(foo, kdims=None):
    return lambda datasets: hv.HoloMap({
        k: foo(ds)
        for k, ds in datasets.items()
    }, kdims=kdims).collate()

def my_hex(ds):
    return grid_diag_vars( 
        lambda ds, kdims: hv.Overlay([hv.HexTiles(ds, kdims)]),
        lambda ds, a: hv.Overlay([hv.Histogram(np.histogram(ds[a].values, bins=20))])
    )(ds).opts(
        opts.HexTiles(colorbar=True, logz=True, gridsize=20),
        opts.Overlay(width=180, height=180)
    )

my_hex(data.to_dataframe())


### The link is there, but looks pretty hard to train on
## Is the data just noisy?
http://scipy-lectures.org/packages/scikit-image/auto_examples/plot_filter_coins.html

In [ ]:
from skimage import filters
from skimage import restoration
from toolz import valmap

smoothings = {
    'none': data,
    'gauss': data.map(lambda x: filters.gaussian(x, sigma=1.)),
    'chambolle': data.map(lambda x: restoration.denoise_tv_chambolle(x, weight=2.)),
    'median': data.map(lambda x: filters.median(x, np.ones((3, 3)) )),
}

map_datasets(layout_vars(plot_image), 'denoising')(
    valmap(lambda ds: ds[{'x': slice(400, 450), 'y': slice(100, 150)}],
           smoothings)
)


In [ ]:
map_datasets(my_hex, 'denoising')(valmap(xr.Dataset.to_dataframe, smoothings))

### Does not look better
- If anything, we need to reversee the projection, find original resolution for every variable, then denoise...
- Maybe we can't just relate albedo to elements without context?
  - Get context with a small convolution!
- Or maybe it will work well enough?

## Let's prepare a test split
### The simplest way to manage data is to finish the cutout

In [ ]:
# Test patch size
xx, yy = data.x[(data.albedo==0).any('y')], data.y[(data.albedo==0).any('x')]
[int(x) for x in (xx.min(), xx.max(), yy.min(), yy.max(), data.x.max(), data.y.max())]

In [ ]:
vx_min, vx_max, vy_min, vy_max, x_max, y_max = [300, 430, 140, 270, 720, 360]

In [ ]:
validation_data = data[{'x': slice(vx_min, vx_max), 'y': slice(vy_min, vy_max)}]
validation_data

In [ ]:
# test_data = data.roll(roll_coords=True, y=-vy_min)[{'x': slice(vx_min, vx_max), 'y': slice(vy_max - vy_min, None)}].roll(roll_coords=True, y=vy_min)
test_data = data.where(
    np.logical_and(
        np.logical_and(data.x >= vx_min, data.x < vx_max),
        np.logical_or(data.y >= vy_max, data.y < vy_min)
    )
)
test_data

In [ ]:
# train_data = data.roll(roll_coords=True, x=-vx_min)[{'x': slice(vx_max-vx_min, None)}].roll(roll_coords=True, x=vx_min)
train_data = data.where(
    np.logical_or(data.x < vx_min, data.x >= vx_max)
)
train_data

### Plot train set

In [ ]:
def layout_datasets(foo, kdims=None):
    return lambda datasets: hv.NdLayout({
        k: foo(ds)
        for k, ds in datasets.items()
    }, kdims=kdims)
    

def map_variables(foo):
    return lambda ds: hv.HoloMap({
        k: foo(ds[k])
        for k in ds
    }, kdims='variables')


layout_datasets(map_variables(plot_image), 'dataset')({
    'full': data,
    'train': train_data,
    'test': test_data,
    'validation': validation_data,
})

In [ ]:
train_data = train_data.dropna('x', 'all')
test_data = test_data.dropna('x', 'all').dropna('y', 'all')

In [ ]:
map_datasets(my_hex, 'dataset')(valmap(xr.Dataset.to_dataframe, {
    'full': data,
    'train': train_data,
    'test': test_data,
    # 'validation': validation_data, # all-zeros break hex?
}))

### Statistically uneven, but so is the validation rectangle!

## Make trivial linear model

### Train

In [ ]:
linear_models = {k: LinearRegression() for k in 'abcd'}
linear_data = data.copy()
for k, m in linear_models.items():
    m.fit(train_data.albedo.data.reshape(-1, 1), train_data[k].data.reshape(-1, 1))
    prediction = m.predict(linear_data.albedo.data.reshape(-1, 1))
    linear_data[k] = (('y', 'x'), prediction.reshape(linear_data[k].shape))

In [ ]:
# Linear model is linear!
hv.Layout([hv.HexTiles(data.to_dataframe(), ['albedo', element]) * hv.Points(linear_data.to_dataframe().sample(100), ['albedo', element]) for element in 'abcd'])

In [ ]:
map_datasets(layout_vars(plot_image), 'model')\
({'original': data, 'linear': linear_data})\
.opts(opts.Image(width=len(data.x), height=len(data.y), colorbar=True))

### Score

In [ ]:
linear_score = pd.DataFrame([
    {
        'score': mean_squared_error(
            ds[el].data.reshape(-1, 1),
            linear_models[el].predict(ds.albedo.data.reshape(-1, 1))
        ),
        'dataset': ds_k,
        'element': el,
        'model': 'linear'
    }
    for ds_k, ds in [('test', test_data), ('train', train_data)]
    for el in 'abcd'
])
linear_score

In [ ]:
score.groupby(['dataset', 'model']).sum()

- worse than the 422.51792 of the original example solution
  - because of the less random train/test split
  - but maybe *more* representative of the expected validation result

## Non-trivial 1-d model

In [ ]:
from xgboost import XGBRegressor
xgb_models = {k: XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100, seed=13, n_jobs=-1) for k in 'abcd'}
xgb_data = data.copy()
for k, m in xgb_models.items():
    m.fit(train_data.albedo.data.reshape(-1, 1), train_data[k].data.reshape(-1))
    prediction = m.predict(xgb_data.albedo.data.reshape(-1, 1))
    xgb_data[k] = (('y', 'x'), prediction.reshape(xgb_data[k].shape))

In [ ]:
hv.Layout([hv.HexTiles(data.to_dataframe(), ['albedo', element]) * hv.Points(xgb_data.to_dataframe().sample(1000), ['albedo', element]) for element in 'abcd'])

In [ ]:
xgb_score = pd.DataFrame([
    {
        'score': mean_squared_error(
            ds[el].data.reshape(-1, 1),
            xgb_models[el].predict(ds.albedo.data.reshape(-1, 1))
        ),
        'dataset': ds_k,
        'element': el,
        'model': 'xgb'
    }
    for ds_k, ds in [('test', test_data), ('train', train_data)]
    for el in 'abcd'
])
xgb_score

In [ ]:
map_datasets(layout_vars(plot_image), 'model')\
({'original': data, 'xgb': xgb_data})\
.opts(opts.Image(width=len(data.x), height=len(data.y), colorbar=True))

### Small convolutional network

In [ ]:
import tensorflow as tf
from tensorflow import keras
def create_model():
    inputs = keras.Input(shape=(None, None, 1))
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=3, activation='elu')
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=3, activation='elu')
    conv3 = keras.layers.Conv2D(filters=128, kernel_size=3, activation='elu')
    conv4 = keras.layers.Conv2D(filters=4, kernel_size=3, activation='elu')
    model = keras.Model(inputs, conv4(conv3(conv2(conv1(inputs)))))
    model.compile(loss='MSE', optimizer='adam', metrics=['MSE'])
    return model

model_conv = keras.wrappers.scikit_learn.KerasRegressor(build_fn=create_model, epochs=512, batch_size=1, verbose=0)

In [ ]:
def prep_conv(ds, pad=4):
    X = ds.albedo.transpose().data
    X = np.pad(X, [(0,), (pad,)], 'wrap')
    X = np.pad(X, [(pad,), (0,)], 'edge')[None, ..., None]
    Y = ds[list('abcd')].to_array().transpose().data[None, ...]
    return X, Y
print([x.shape for x in prep_conv(data)])

In [ ]:
history = model_conv.fit(*prep_conv(train_data))

In [ ]:
pd.DataFrame(history.history['MSE']).plot()

In [ ]:
conv_data = data.copy()
X, _ = prep_conv(data)
pred_conv = model_conv.predict(X)
print(pred_conv.shape)
for i, k in enumerate('abcd'):
    prediction = pred_conv[..., i].transpose()
    conv_data[k] = (('y', 'x'), prediction)

In [ ]:
pred_test = model_conv.predict(X_test)
pred_test.shape

In [ ]:
conv_score = pd.DataFrame([
    {
        'score': mean_squared_error(
            prep_conv(ds)[1][0, ..., i],
            model_conv.predict(prep_conv(ds)[0])[..., i]
        ),
        'dataset': ds_k,
        'element': el,
        'model': 'conv'
    }
    for ds_k, ds in [('test', test_data), ('train', train_data)]
    for i, el in enumerate('abcd')
])
conv_score

In [ ]:
hv.Layout([hv.HexTiles(data.to_dataframe(), ['albedo', element]) * hv.Points(conv_data.to_dataframe().sample(1000), ['albedo', element]) for element in 'abcd'])

In [ ]:
map_datasets(layout_vars(plot_image), 'model')\
({'original': data, 'conv': conv_data})\
.opts(opts.Image(width=len(data.x), height=len(data.y), colorbar=True))

# Deliverables

* Google Colab Jupyter Notebook showing your solution along with the final model score More details regarding the format of the notebook can be found in the sample Google Colab notebook provided for this challenge.  
* A txt file for each element containing your predictions on the test data. Format should be: x_coordinate, y_coordinate, predicted_value. Put name of element in file. An example is provided.
* The final trained model including the model architecture and the trained weights (For example: HDF5 file, .pb file, .pt, .sav file, etc.). You are free to choose Machine Learning Framework of your choice.
* Example submissions can be found https://drive.google.com/drive/folders/1EsqNLc5DzCsaJuvSTYF85gMS5PTVell4?usp=sharing

## Save models
Other formats can be used

In [ ]:
pickle.dump(element_A_model, open('./element_A_model.sav', 'wb'))
pickle.dump(element_B_model, open('./element_B_model.sav', 'wb'))
pickle.dump(element_C_model, open('./element_C_model.sav', 'wb'))
pickle.dump(element_D_model, open('./element_D_model.sav', 'wb'))

## Save results

In [ ]:
# Map that only looks at test data (train pixels = -1e6 to ensure no overlap with predictions)
reconstructed_A_predictions = np.zeros(len(element_B)) - 1e6
reconstructed_B_predictions = np.zeros(len(element_B)) - 1e6
reconstructed_C_predictions = np.zeros(len(element_B)) - 1e6
reconstructed_D_predictions = np.zeros(len(element_B)) - 1e6

# Fill in predicted values
for (i, index) in enumerate(test_indices):
    reconstructed_A_predictions[index] = element_A_pred[i][0]
    reconstructed_B_predictions[index] = element_B_pred[i][0]
    reconstructed_C_predictions[index] = element_C_pred[i][0]
    reconstructed_D_predictions[index] = element_D_pred[i][0]
        
# Reshape into images
reconstructed_A_predictions = reconstructed_A_predictions.reshape(data_shape)
reconstructed_B_predictions = reconstructed_B_predictions.reshape(data_shape)
reconstructed_C_predictions = reconstructed_C_predictions.reshape(data_shape)
reconstructed_D_predictions = reconstructed_D_predictions.reshape(data_shape)

# Get coordinates of pixel
xs, ys = [], []
A_values, B_values, C_values, D_values = [], [], [], []

for i in range(len(reconstructed_A_predictions)):
    for j in range(len(reconstructed_A_predictions[i])):
        if reconstructed_A_predictions[i][j] == -1e6:
            continue
        xs.append(i)
        ys.append(j)
        A_values.append(reconstructed_A_predictions[i][j])
        B_values.append(reconstructed_B_predictions[i][j])
        C_values.append(reconstructed_C_predictions[i][j])
        D_values.append(reconstructed_D_predictions[i][j])

        
# Save
with open('./element_A_predictions.txt', 'w') as f:
    for i in range(len(xs)):
        this_str = '%i, %i, %f \n' % (xs[i], ys[i], A_values[i])
        f.write(this_str)
        
with open('./element_B_predictions.txt', 'w') as f:
    for i in range(len(xs)):
        this_str = '%i, %i, %f \n' % (xs[i], ys[i], A_values[i])
        f.write(this_str)
        
with open('./element_C_predictions.txt', 'w') as f:
    for i in range(len(xs)):
        this_str = '%i, %i, %f \n' % (xs[i], ys[i], A_values[i])
        f.write(this_str)
        
with open('./element_D_predictions.txt', 'w') as f:
    for i in range(len(xs)):
        this_str = '%i, %i, %f \n' % (xs[i], ys[i], A_values[i])
        f.write(this_str)


# Optional Diagnostic Plots

### Reconstruct map using predictions

In [ ]:
# Entire map
reconstructed_element_A = element_A.copy()
reconstructed_element_B = element_B.copy()
reconstructed_element_C = element_C.copy()
reconstructed_element_D = element_D.copy()

for (i, index) in enumerate(test_indices):
    
    # If the predicted pixel is unmasked, put it in the new map
    if index in flat_unmasked_indices[0]:
        reconstructed_element_A[index] = element_A_pred[i][0]
        reconstructed_element_B[index] = element_B_pred[i][0]
        reconstructed_element_C[index] = element_C_pred[i][0]
        reconstructed_element_D[index] = element_D_pred[i][0]

    else:
        reconstructed_element_A[index] = 0
        reconstructed_element_B[index] = 0
        reconstructed_element_C[index] = 0
        reconstructed_element_D[index] = 0 
        
# Make withheld region
reconstructed_element_A = np.ma.MaskedArray(reconstructed_element_A, flat_mask)
reconstructed_element_B = np.ma.MaskedArray(reconstructed_element_B, flat_mask)
reconstructed_element_C = np.ma.MaskedArray(reconstructed_element_C, flat_mask)
reconstructed_element_D = np.ma.MaskedArray(reconstructed_element_D, flat_mask)

# Reshape into image
reconstructed_element_A = reconstructed_element_A.reshape(data_shape)
reconstructed_element_B = reconstructed_element_B.reshape(data_shape)
reconstructed_element_C = reconstructed_element_C.reshape(data_shape)
reconstructed_element_D = reconstructed_element_D.reshape(data_shape)

### Plot reconstructed map (including training data)

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(26, 14))

im = axs[0, 0].imshow(reconstructed_element_A)
axs[0, 0].set_title('Reconstructed Element A')
# Set Colorbars
divider = make_axes_locatable(axs[0, 0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

im = axs[0, 1].imshow(reconstructed_element_B)
axs[0, 1].set_title('Reconstructed Element B')
divider = make_axes_locatable(axs[0, 1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

im = axs[1, 0].imshow(reconstructed_element_C)
axs[1, 0].set_title('Reconstructed Element C')
divider = make_axes_locatable(axs[1, 0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

im = axs[1, 1].imshow(reconstructed_element_D)
axs[1, 1].set_title('Reconstructed Element D')
divider = make_axes_locatable(axs[1, 1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.show()

### Plot difference between predicted and actual maps

In [ ]:
# Plot difference between prediction and true
difference_A = reconstructed_element_A - np.ma.MaskedArray(element_A_data, flat_mask)
difference_B = reconstructed_element_B - np.ma.MaskedArray(element_B_data, flat_mask)
difference_C = reconstructed_element_C - np.ma.MaskedArray(element_C_data, flat_mask)
difference_D = reconstructed_element_D - np.ma.MaskedArray(element_D_data, flat_mask)

fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(26, 14))

im = axs[0, 0].imshow(difference_A, cmap='RdBu_r')
axs[0, 0].set_title('Predicted Element A - Element A')
# Set Colorbars
divider = make_axes_locatable(axs[0, 0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

im = axs[0, 1].imshow(difference_B, cmap='RdBu_r')
axs[0, 1].set_title('Predicted Element B - Element B')
divider = make_axes_locatable(axs[0, 1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

im = axs[1, 0].imshow(difference_C, cmap='RdBu_r')
axs[1, 0].set_title('Predicted Element C - Element C')
divider = make_axes_locatable(axs[1, 0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

im = axs[1, 1].imshow(difference_D, cmap='RdBu_r')
axs[1, 1].set_title('Predicted Element D - Element D')
divider = make_axes_locatable(axs[1, 1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.show()